In [55]:
import sys
if 'myfilechooser' in sys.modules:  
    del sys.modules["myfilechooser"]
from myfilechooser import MyFileChooser
import ipywidgets as wid
fc = MyFileChooser(".", title='Choose file', filter='*.csv', layout=wid.Layout(width='auto'))
display(fc)

MyFileChooser(path='/Users/maurizio/HELP/prova', filename='()', title='Choose file', show_hidden=False, select…

('myfilechooser.py',) ()
('myfilechooser.py', 'prova.ipynb') ()
D:  ('myfilechooser.py', 'prova.ipynb')
() ('myfilechooser.py', 'prova.ipynb')
() ('myfilechooser.py', 'prova.ipynb')
('Kidney_CCcfs_0.csv',) ('myfilechooser.py', 'prova.ipynb')
('Kidney_CCcfs_0.csv', 'Kidney_CCcfs_1.csv') ('myfilechooser.py', 'prova.ipynb')
D:  ()
D:  ()
D:  ()
